# 2.2 数据预处理
- **目录**
  - 2.2.1 读取数据集
  - 2.2.2 处理缺失值
  - 2.2.3 转换为张量格式
<br><br>
- 通过Pandas对数据进行预处理并和Pytoch张量兼容。

## 2.2.1 读取数据集

-  创建一个人工数据集，并存储在CSV（Comma-Separated Values, 逗号分隔值）文件`../data/house_tiny.csv`中。
  - 数据集有4行3列。
  - 3个字段：包括房间数量（“NumRooms”）、巷子类型（“Alley”）和房屋价格（“Price”）。


In [7]:
import os
os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # 列名
    f.write('NA,Pave,127500\n')  # 每行表示一个数据样本
    f.write('2,NA,106000\n') # NA表示Not Available之意：缺失值或不可用的值
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

- 加载原始数据集


In [8]:
# 如果没有安装pandas，只需取消对以下行的注释来安装pandas
# !pip install pandas
import pandas as pd
data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


-------
- **说明：NaN与NA的区别与联系**
  - NaN的涵义：
    -  NaN即"Not a Number"之意，是Numpy引入的一个特殊的浮点数值,用于表示未定义或不可表示的数值。
    - NaN是由 IEEE 754 浮点数标准定义的,主要有以下特点:
      - NaN是一种特殊的浮点数值,不等于任何其他数值,包括它自己。例如, `NaN == NaN`的结果是`False`。
      - 任何涉及 NaN 的数学运算都会产生 NaN 作为结果。例如, `1 + NaN`、`NaN * 5`、`NaN / 3` 等都会得到 NaN。
      - NaN的比较操作也是特殊的。任何涉及 NaN 的比较操作都会返回 `False`。例如, `NaN < 5`、`NaN >= NaN` 都会返回 `False`。
      - NaN 主要用于表示数据中的缺失值或无效值。在 Python 中,当数据中出现无法表示的值时,通常会用 NaN 来代替。
      - 即使在整数列中引入 `NaN`，该列也会被提升为浮点数类型。
    - 在 Python 中,NaN 通常出现在以下情况:
      - 使用 `float('nan')` 或 `numpy.nan` 创建 NaN 值。
      - 在数据导入(如 CSV 文件)或数据计算过程中出现无法表示的值。
      - 使用 `pandas` 库处理数据时,缺失值通常被表示为 NaN。
    - 处理 NaN 值是数据预处理中的一个重要步骤,常见的方法包括插补、删除等。了解 NaN 的特性有助于更好地管理和处理数据中的缺失值。
  - NA的涵义：
    - NA即"Not Available"之意。
    - 是Pandas 1.0.0引入的新缺失值标识符，旨在统一和改进缺失值处理。
    - NA是一个通用的缺失值标识符，可以用于任何数据类型（整数、浮点数、对象、字符串、布尔等）。
  - NaN与NA的区别与联系:
    - NaN是IEEE 754浮点数标准定义的一种特殊的浮点数值,用于表示未定义或不可表示的数值。而NA是一个更广义的概念,可以表示任何类型的缺失值。
    - NaN主要出现在浮点数计算中,而 NA 可以出现在任何数据类型中,包括整数、字符串等。
    - 在Python中,NaN 通常由 `numpy.nan` 表示,而 NA 通常由 `pandas.NA` 表示。
    - Pandas 处理缺失值时，NaN 和 NA 可以互换。例如，pd.isna() 函数可以检测到两者。
    - 在Pandas中,NA 被用作缺失值的统一标记。当 Pandas 数据结构(Series 或 DataFrame)中的某个单元格没有值时,它会被标记为 `pandas.NA`。这种统一的缺失值标记使得数据分析和处理更加方便和可靠。
    - 例如,在 Pandas 中使用 `df.isna()` 可以检查数据框中哪些单元格包含缺失值,使用 `df.fillna()` 可以填充缺失值。
    - 在早期版本的 Pandas 中，主要使用 NaN。在较新的版本中，推荐使用 NA，因为它提供了更强大和一致的缺失值处理能力。
    - 如需保持列的原始数据类型（例如整数），可以使用 NA 而不是 NaN。

In [9]:
import pandas as pd
import numpy as np

# 使用 NaN
df_nan = pd.DataFrame({
    'A': [1, 2, np.nan],
    'B': [4, np.nan, 6]
})

# 使用 NA
df_na = pd.DataFrame({
    'A': [1, 2, pd.NA],
    'B': [4, pd.NA, 6]
})

print(df_nan)
print(df_na)

# 检查缺失值
print(df_nan.isna())
print(df_na.isna())


     A    B
0  1.0  4.0
1  2.0  NaN
2  NaN  6.0
      A     B
0     1     4
1     2  <NA>
2  <NA>     6
       A      B
0  False  False
1  False   True
2   True  False
       A      B
0  False  False
1  False   True
2   True  False


-----------

## 2.2.2 处理缺失值
- Python中的“NaN”项代表缺失值。
- 处理缺失的数据，典型的方法包括**插值法**和**删除法**：
  - 插值法用一个替代值补充缺失值。
  - 删除法则直接忽略缺失值。
  - 注意：上述所谓插值法，准确讲应该是"**插补法**"英语单词为Imputation，而不是插值Interpolation,后者有专门的涵义。
- 其他比较重要处理方法可以使用sklearn的插补法
- 示例：
  - 通过位置索引`iloc`，将`data`分成`inputs`和`outputs`。
  - `inputs`为`data`的前两列，`outputs`为`data`的最后一列。
  - 对于`inputs`中缺少的数值，使用同一列的均值替换“NaN”项。


In [11]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = inputs.fillna(inputs.mean(numeric_only=True))
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN


- 对于`inputs`中的**类别值**或**离散值**，可将“NaN”视为一个类别：
  - “巷子类型”（“Alley”）列只接受两种类型的类别值“Pave”和“NaN”。
  - pandas可以自动将此列转换为两列“Alley_Pave”和“Alley_nan”。
  - 巷子类型为“Pave”的行会将“Alley_Pave”的值设置为1，“Alley_nan”的值设置为0。
  - 缺少巷子类型的行会将“Alley_Pave”和“Alley_nan”分别设置为0和1。


In [12]:
inputs = pd.get_dummies(inputs, dummy_na=True)
inputs = inputs.astype(int)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0         3           1          0
1         2           0          1
2         4           0          1
3         3           0          1


In [13]:
inputs['Alley_Pave']

0    1
1    0
2    0
3    0
Name: Alley_Pave, dtype: int32

- 在`scikit-learn`库中，提供了不同的方法来处理数据中的缺失值。
  - 包括`SimpleImputer`、`KNNImputer`和`IterativeImputer`等。
- 示例：
  - 使用`SimpleImputer`

In [14]:
data.iloc[:,0]

0    NaN
1    2.0
2    4.0
3    NaN
Name: NumRooms, dtype: float64

In [15]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

# 创建一个包含空值的数据框（原示例数据）
df = pd.DataFrame(data.iloc[:,0])

# 使用SimpleImputer处理缺失值
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputed_df = imputer.fit_transform(df)

# 将处理后的NumPy数组转换回DataFrame
imputed_df = pd.DataFrame(imputed_df, columns=df.columns)
imputed_df

,NumRooms
0,3.0
1,2.0
2,4.0
3,3.0


In [17]:
from sklearn.impute import SimpleImputer
SimpleImputer?

Init signature:
SimpleImputer(
    *,
    missing_values=nan,
    strategy='mean',
    fill_value=None,
    verbose='deprecated',
    copy=True,
    add_indicator=False,
    keep_empty_features=False,
)
Docstring:     
Univariate imputer for completing missing values with simple strategies.

Replace missing values using a descriptive statistic (e.g. mean, median, or
most frequent) along each column, or using a constant value.

Read more in the :ref:`User Guide <impute>`.

.. versionadded:: 0.20
   `SimpleImputer` replaces the previous `sklearn.preprocessing.Imputer`
   estimator which is now removed.

Parameters
----------
missing_values : int, float, str, np.nan, None or pandas.NA, default=np.nan
    The placeholder for the missing values. All occurrences of
    `missing_values` will be imputed. For pandas' dataframes with
    nullable integer dtypes with missing values, `missing_values`
    can be set to either `np.nan` or `pd.NA`.

strategy : str, default='mean'
    The imputation s

## 2.2.3 转换为张量格式

- 现在`inputs`和`outputs`中的所有条目都是数值类型，它们可以转换为张量格式。
- 通过`torch.tensor()`函数转换成张量。

In [18]:
import torch
X, y = torch.tensor(inputs.values), torch.tensor(outputs.values)
X, y

(tensor([[3, 1, 0],
         [2, 0, 1],
         [4, 0, 1],
         [3, 0, 1]], dtype=torch.int32),
 tensor([127500, 106000, 178100, 140000]))

## 小结

* `pandas`软件包是Python中常用的数据分析工具，`pandas`可以与张量兼容。
* 用`pandas`处理缺失的数据时，我们可根据情况选择用插值法和删除法。
* sklearn包中的插补器工具功能更加强大，在数据预处理时需要掌握。


